In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, date

import json 
import geopandas as gpd

from shapely import wkt, Polygon
from shapely.wkt import loads
from shapely.geometry import box, Polygon

import folium
from folium.plugins import HeatMap

import ipywidgets as widgets

from IPython.display import display
from traitlets import Unicode, validate


In [2]:

def load_satellite_data(file_path: str, company_name: str) -> pd.DataFrame:
    # Jason to DF
    with open(file_path, 'r') as f:
        data = json.load(f)
    df = pd.DataFrame(data)
    df['company'] = company_name
    return df

def process_blacksky_planet_data(df: pd.DataFrame, company_name: str, date_key: str) -> pd.DataFrame:
    # blacksky_planet process and create polygons
    df['company'] = company_name
    df['coordinates'] = df['geometry'].apply(lambda x: x['coordinates'][0])
    df['type'] = df['geometry'].apply(lambda x: x['type'])
    df['date'] = df['properties'].apply(lambda x: x[date_key]['$date'])
    df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d %H:%M:%S')
    df = df[(df['date'] > '2022-01-01 00:00:00') & (df['type'] == 'Polygon')]
    df['geometry'] = df['coordinates'].apply(Polygon)
 
    return df[['company', 'date', 'coordinates', 'type','geometry']]


def process_capella_data(df: pd.DataFrame) -> pd.DataFrame:
    # capella process
    df['type'] = df['geometry'].apply(lambda x: x.geom_type)
    df['coordinates'] = df['geometry'].apply(lambda x: list(x.exterior.coords))
    df['date'] = df['datetime']

    df = df[df['date'].str.contains(r'^[0-9]{4}/[0-9]{2}/[0-9]{2} [0-9]{2}:[0-9]{2}:[0-5][0-9]\.[0-9]{3}\+00$')]

    df['date'] = pd.to_datetime(df['date'], format='%Y/%m/%d %H:%M:%S.%f+00')
    df['date'] = pd.to_datetime(df['date'])
    df = df[(df['date'] > '2022-01-01 00:00:00') & (df['type'] == 'Polygon')]

    return df[['company', 'date', 'coordinates', 'type', 'geometry']]


def create_israel_polygon() -> Polygon:
    # israel borders polygon
    return Polygon([
        (35.1073608, 33.0958637),
        (35.0958250, 33.0948344),
        (34.9420166, 32.8288271),
        (34.8782957, 32.5607035),
        (34.7244874, 32.0258327),
        (34.1971435, 31.3556223),
        (34.9244384, 29.3716705),
        (35.5561521, 31.3471808),
        (35.5687866, 32.6322884),
        (35.9450683, 32.8701446),
        (35.7857666, 33.3232990),
        (35.5687866, 33.2865818),
        (35.5303344, 33.1326609),
        (35.1073608, 33.0958637)
    ])

def filter_by_israel_bounds(df: gpd.GeoDataFrame, israel_polygon: Polygon) -> gpd.GeoDataFrame:
    # all intersects polygons
    return df[
        df['geometry'].notnull() &
        df['geometry'].apply(lambda x: x is not None and x.intersects(israel_polygon))
    ]

def main():
    #load data 
    df_blacksky = load_satellite_data('Catalogs/blacksky-archive.shape.json', 'BlackSky')
    df_planet = load_satellite_data('Catalogs/planet-archive.features.json', 'Planet')
    df_cappela = gpd.read_file('Catalogs/capella_archive_export_full.geojson')
    df_cappela['company'] = 'Capella'

    #prosses data
    df_blacksky_processed = process_blacksky_planet_data(df_blacksky, 'BlackSky', 'datetime')
    df_planet_processed = process_blacksky_planet_data(df_planet, 'Planet', 'acquired')
    df_cappela_processed = process_capella_data(df_cappela)

    # concat all DF
    df_all = pd.concat([df_blacksky_processed, df_planet_processed,df_cappela_processed], ignore_index=True)
    df_all['date'] = pd.to_datetime(df_all['date'])


    # all polygons inside israel border
    israel_polygon = create_israel_polygon()
    df_israel = filter_by_israel_bounds(gpd.GeoDataFrame(df_all, geometry='geometry'), israel_polygon)

    return df_israel

df_israel = main()


C:\Users\user\AppData\Local\Temp\ipykernel_21972\2409086502.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['geometry'] = df['coordinates'].apply(Polygon)
C:\Users\user\AppData\Local\Temp\ipykernel_21972\2409086502.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['geometry'] = df['coordinates'].apply(Polygon)
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyW

In [15]:

# widget to set dates and update map
pick_start = widgets.DatePicker(value=datetime(2023, 1, 1), description='Start Date', dateFormat='dd/mm/yy')
pick_end = widgets.DatePicker(value=datetime.today(), description='End Date', dateFormat='dd/mm/yy')
update_button = widgets.Button(description="Update Map")
center_layout = widgets.Layout(display='flex', align_items='center', width='100%')
widgets_HBox = widgets.HBox(children=[pick_start, pick_end, update_button], layout=center_layout)
display(widgets_HBox)

# Grid cell size
cell_size = 0.05

# Function to create heatmap data
def create_heatmap_data(geo_df, cell_size):
    bounds = geo_df.total_bounds
    x_coords = np.arange(bounds[0], bounds[2] + cell_size, cell_size)
    y_coords = np.arange(bounds[1], bounds[3] + cell_size, cell_size)
    
    heat_data = []
    for x in x_coords:
        for y in y_coords:
            cell = box(x, y, x + cell_size, y + cell_size)
            intersecting_polygons = geo_df[geo_df.geometry.intersects(cell)]
            
            if intersecting_polygons.shape[0] > 1:
                companies = ", ".join(intersecting_polygons['company'].unique()) or "No Companies"
                popup_msg = f"Companies: {companies}<br>Number of Polygons: {intersecting_polygons.shape[0]}"
                centroid = intersecting_polygons.geometry.centroid.union_all().centroid
                heat_data.append([centroid.y, centroid.x, intersecting_polygons.shape[0], popup_msg])
    
    return heat_data

# Function to update the heatmap
def update_heatmap(button):
    start_date = pd.to_datetime(pick_start.value)
    end_date = pd.to_datetime(pick_end.value)
    
    filtered_df = df_israel[(df_israel['date'] >= start_date) & (df_israel['date'] <= end_date)]
    heat_data = create_heatmap_data(filtered_df, cell_size)
    
    # Create map centered around Israel
    heatmap = folium.Map(location=[31.0461, 34.8516], zoom_start=7)
    
    # Create a layer for polygons (not visible by default)
    polygon_layer = folium.FeatureGroup(name="Polygons Layer", show=False)
    
    # Add polygons to the polygon layer
    for _, row in filtered_df.iterrows():
        polygon = folium.Polygon(
            locations=[(coord[1], coord[0]) for coord in row['geometry'].exterior.coords],
            color='blue',
            weight=1.5,
            opacity=0.4,
            popup=row['company']
        )
        polygon.add_to(polygon_layer)
    
    polygon_layer.add_to(heatmap)
    
    heatmap_layer = folium.FeatureGroup(name="Heat")
    
    HeatMap(
        data=[(lat, lon, count) for lat, lon, count, _ in heat_data],
        radius=15, gradient={0.4: 'blue', 0.65: 'lime', 1: 'red'},
        max_opacity=0.8, min_opacity=0.3
    ).add_to(heatmap_layer)
    
    heatmap_layer.add_to(heatmap)
    
    # Add popups for each grid cell (optional)
    for lat, lon, _, popup_msg in heat_data:
        folium.Marker(location=[lat, lon], popup=popup_msg, icon=folium.DivIcon(html='')).add_to(heatmap)
    
    # Add Layers Control
    folium.LayerControl().add_to(heatmap)
    
    # Save and display map
    heatmap.save('templates/heatmap_with_grid_and_polygons_layer.html')
    display(heatmap)


# update button
update_button.on_click(update_heatmap)
